In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import json
import csv
import random
from datetime import datetime, timedelta

# Inicializar a sessão do Spark
spark = SparkSession.builder \
    .appName("EmpresaYAnaliseDeDados") \
    .getOrCreate()

24/11/24 16:21:22 WARN Utils: Your hostname, Mac-mini-de-Valter.local resolves to a loopback address: 127.0.0.1; using 192.168.0.100 instead (on interface en0)
24/11/24 16:21:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/24 16:21:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [28]:
# Gerar dados de vendas
def gerar_dados_vendas():
    produtos = ["Notebook", "Smartphone", "Tablet", "VideoGame", "Fone de Ouvido", "Smartwatch"]
    with open("dados_vendas.csv", "w", newline='') as arquivo:
        escritor = csv.writer(arquivo)
        escritor.writerow(["id_venda", "id_produto", "produto", "quantidade", "preco", "data"])
        for i in range(1000):
            id_produto = random.randint(1, 6)
            escritor.writerow([
                i,
                id_produto,
                produtos[id_produto - 1],
                random.randint(1, 5),
                "{:.2f}".format(random.uniform(100, 1000)),
                (datetime.now() - timedelta(days=random.randint(1, 365))).strftime("%Y-%m-%d")
            ])

# Gerar dados de comportamento do cliente
def gerar_dados_comportamento_cliente():
    acoes = ["visualizacao", "adicionar_ao_carrinho", "compra", "avaliacao"]
    dados = []
    for i in range(1000):
        dados.append({
            "id_cliente": random.randint(1, 100),
            "id_produto": random.randint(1, 6),
            "acao": random.choice(acoes),
            "data": (datetime.now() - timedelta(days=random.randint(1, 365))).strftime("%Y-%m-%d")
        })
    with open("dados_comportamento_cliente.json", "w") as arquivo:
        json.dump(dados, arquivo)

# Gerar dados de estoque
def gerar_dados_estoque():
    produtos = ["Notebook", "Smartphone", "Tablet", "Smartwatch", "Fone de Ouvido", "Smartwatch"]
    with open("dados_estoque.csv", "w", newline='') as arquivo:
        escritor = csv.writer(arquivo)
        escritor.writerow(["id_produto", "produto", "quantidade"])
        for i in range(6):
            escritor.writerow([
                i,
                produtos[i-1],
                random.randint(0, 1000),
                datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            ])

In [29]:
# Gerar todos os dados
gerar_dados_vendas()
gerar_dados_comportamento_cliente()
gerar_dados_estoque()

In [35]:
# Carregar dados de vendas
df_vendas = spark.read.csv("dados_vendas.csv", header=True, inferSchema=True)
print("Esquema dos dados de vendas:")
print(df_vendas.printSchema())

# Carregar dados de comportamento do cliente
df_comportamento_cliente = spark.read.json("dados_comportamento_cliente.json")
print("Esquema dos dados de comportamento do cliente:")
print(df_comportamento_cliente.printSchema())

# Carregar dados de estoque
df_estoque = spark.read.csv("dados_estoque.csv", header=True, inferSchema=True)
print("Esquema dos dados de estoque:")
print(df_estoque.printSchema())

Esquema dos dados de vendas:
root
 |-- id_venda: integer (nullable = true)
 |-- id_produto: integer (nullable = true)
 |-- produto: string (nullable = true)
 |-- quantidade: integer (nullable = true)
 |-- preco: double (nullable = true)
 |-- data: date (nullable = true)

None
Esquema dos dados de comportamento do cliente:
root
 |-- acao: string (nullable = true)
 |-- data: string (nullable = true)
 |-- id_cliente: long (nullable = true)
 |-- id_produto: long (nullable = true)

None
Esquema dos dados de estoque:
root
 |-- id_produto: integer (nullable = true)
 |-- produto: string (nullable = true)
 |-- quantidade: integer (nullable = true)

None


In [32]:
# Processar dados de vendas
resumo_vendas1 = df_vendas \
    .groupBy("id_produto", "produto") \
    .agg(
        sum("quantidade").alias("quantidade_total_vendida"),
        sum(col("quantidade") * col("preco")).alias("faturamento_total"),
    ) \
     .withColumn("faturamento_total", format_number("faturamento_total", 2)) \
    .orderBy("faturamento_total", ascending=False)
    # .orderBy("id_produto", ascending=True)

print("Resumo de vendas 1:")
resumo_vendas1.show()

resumo_vendas2 = df_vendas \
    .groupBy("produto") \
    .agg(
        sum("quantidade").alias("quantidade_total_vendida"),
        avg("preco").alias("preco_medio"),
        min("preco").alias("preco_minimo"),
        max("preco").alias("preco_maximo")
    )

print("Resumo de vendas 2:")
resumo_vendas2.show()

Resumo de vendas 1:
+----------+--------------+------------------------+-----------------+
|id_produto|       produto|quantidade_total_vendida|faturamento_total|
+----------+--------------+------------------------+-----------------+
|         2|    Smartphone|                     515|       292,671.50|
|         4|     VideoGame|                     513|       288,045.00|
|         6|    Smartwatch|                     517|       272,128.73|
|         1|      Notebook|                     480|       268,600.12|
|         3|        Tablet|                     504|       258,027.52|
|         5|Fone de Ouvido|                     436|       234,501.29|
+----------+--------------+------------------------+-----------------+

Resumo de vendas 2:
+--------------+------------------------+-----------------+------------+------------+
|       produto|quantidade_total_vendida|      preco_medio|preco_minimo|preco_maximo|
+--------------+------------------------+-----------------+------------+-----

In [41]:
# Processar dados de comportamento do cliente
resumo_comportamento_cliente = df_comportamento_cliente \
    .groupBy("id_produto", "acao") \
    .count() \
    .groupBy("id_produto") \
    .agg(
        sum(when(col("acao") == "visualizacao", col("count")).otherwise(0)).alias("visualizacoes"),
        sum(when(col("acao") == "adicionar_ao_carrinho", col("count")).otherwise(0)).alias("adicoes_ao_carrinho"),
        sum(when(col("acao") == "compra", col("count")).otherwise(0)).alias("compras"),
        sum(when(col("acao") == "avaliacao", col("count")).otherwise(0)).alias("avaliacoes"),
    ) \
    .orderBy("id_produto", ascending=True)

print("Resumo de comportamento do cliente:")
resumo_comportamento_cliente.show()

Resumo de comportamento do cliente:
+----------+-------------+-------------------+-------+----------+
|id_produto|visualizacoes|adicoes_ao_carrinho|compras|avaliacoes|
+----------+-------------+-------------------+-------+----------+
|         1|           44|                 47|     39|        43|
|         2|           41|                 30|     40|        35|
|         3|           51|                 44|     42|        45|
|         4|           48|                 34|     41|        38|
|         5|           52|                 42|     35|        35|
|         6|           53|                 44|     33|        44|
+----------+-------------+-------------------+-------+----------+



In [43]:
# Processar dados de estoque
produtos_baixo_estoque = df_estoque \
    .filter(col("quantidade") < 100) \
    .orderBy("quantidade", ascending=True)

print("Produtos com baixo estoque:")
produtos_baixo_estoque.show()

Produtos com baixo estoque:
+----------+----------+----------+
|id_produto|   produto|quantidade|
+----------+----------+----------+
|         2|Smartphone|        50|
+----------+----------+----------+



In [46]:
# Juntar dados de vendas e estoque
vendas_estoque = resumo_vendas1 \
    .join(df_estoque, resumo_vendas1.id_produto == df_estoque.id_produto) \
    .select(
        resumo_vendas1.id_produto,
        resumo_vendas1.produto,
        resumo_vendas1.quantidade_total_vendida,
        resumo_vendas1.faturamento_total,
        df_estoque.quantidade
    ) \
    .orderBy("id_produto", ascending=True)

print("Vendas e estoque:")
vendas_estoque.show()

Vendas e estoque:
+----------+--------------+------------------------+-----------------+----------+
|id_produto|       produto|quantidade_total_vendida|faturamento_total|quantidade|
+----------+--------------+------------------------+-----------------+----------+
|         1|      Notebook|                     480|       268,600.12|       709|
|         2|    Smartphone|                     515|       292,671.50|        50|
|         3|        Tablet|                     504|       258,027.52|       476|
|         4|     VideoGame|                     513|       288,045.00|       127|
|         5|Fone de Ouvido|                     436|       234,501.29|       539|
+----------+--------------+------------------------+-----------------+----------+



In [48]:
# Calcular produtos mais vendidos com baixo estoque
produtos_mais_vendidos_baixo_estoque = vendas_estoque \
    .filter(col("quantidade_total_vendida") > 100) \
    .filter(col("quantidade") < 100) \
    .orderBy(col("quantidade_total_vendida").desc()) \
    .limit(5)

print("Produtos mais vendidos com baixo estoque:")
produtos_mais_vendidos_baixo_estoque.show()

Produtos mais vendidos com baixo estoque:
+----------+----------+------------------------+-----------------+----------+
|id_produto|   produto|quantidade_total_vendida|faturamento_total|quantidade|
+----------+----------+------------------------+-----------------+----------+
|         2|Smartphone|                     515|       292,671.50|        50|
+----------+----------+------------------------+-----------------+----------+



In [50]:
# Analisar comportamento do cliente para produtos mais vendidos
produtos_top = resumo_vendas1 \
    .orderBy(col("quantidade_total_vendida").desc()) \
    .limit(5)

comportamento_produtos_top = resumo_comportamento_cliente \
    .join(produtos_top, resumo_comportamento_cliente.id_produto == produtos_top.id_produto) \
    .select(
        resumo_comportamento_cliente.id_produto,
        resumo_comportamento_cliente.visualizacoes,
        resumo_comportamento_cliente.adicoes_ao_carrinho,
        resumo_comportamento_cliente.compras,
        resumo_comportamento_cliente.avaliacoes
    ) \
    .orderBy("id_produto", ascending=True)

print("Produtos mais vendidos:")
produtos_top.show()

print("Comportamento do cliente para produtos mais vendidos:")
comportamento_produtos_top.show()

Produtos mais vendidos:
+----------+----------+------------------------+-----------------+
|id_produto|   produto|quantidade_total_vendida|faturamento_total|
+----------+----------+------------------------+-----------------+
|         6|Smartwatch|                     517|       272,128.73|
|         2|Smartphone|                     515|       292,671.50|
|         4| VideoGame|                     513|       288,045.00|
|         3|    Tablet|                     504|       258,027.52|
|         1|  Notebook|                     480|       268,600.12|
+----------+----------+------------------------+-----------------+

Comportamento do cliente para produtos mais vendidos:
+----------+-------------+-------------------+-------+----------+
|id_produto|visualizacoes|adicoes_ao_carrinho|compras|avaliacoes|
+----------+-------------+-------------------+-------+----------+
|         1|           44|                 47|     39|        43|
|         2|           41|                 30|     40|

In [56]:
# Calcular a taxa de conversão (visualizações para compras)
# Adicionamos 0.01 para evitar divisão por zero
taxa_conversao = resumo_comportamento_cliente \
    .withColumn("taxa_conversao", col("compras") / col("visualizacoes") + 0.01) \
    .orderBy("id_produto", ascending=True)

print("Taxa de conversão:")
taxa_conversao.show()

Taxa de conversão:
+----------+-------------+-------------------+-------+----------+------------------+
|id_produto|visualizacoes|adicoes_ao_carrinho|compras|avaliacoes|    taxa_conversao|
+----------+-------------+-------------------+-------+----------+------------------+
|         1|           44|                 47|     39|        43|0.8963636363636364|
|         2|           41|                 30|     40|        35| 0.985609756097561|
|         3|           51|                 44|     42|        45|0.8335294117647059|
|         4|           48|                 34|     41|        38|0.8641666666666666|
|         5|           52|                 42|     35|        35|0.6830769230769231|
|         6|           53|                 44|     33|        44|0.6326415094339622|
+----------+-------------+-------------------+-------+----------+------------------+

